# Calendar App Models Documentation

This document provides detailed information about the models defined in the `calendar` app (`apps/calendar/models.py`).

## 1. TimeStampedModel (Abstract Base Class)

An abstract base class that provides self-updating `created_at` and `updated_at` fields. This is used as a parent class for other models to automatically track creation and modification times.

In [ ]:
class TimeStampedModel(models.Model):
    """Abstract base class that provides self-updating
    ``created_at`` and ``updated_at`` fields."""
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)

    class Meta:
        abstract = True

### Properties:
- **created_at**: `DateTimeField`
  - `auto_now_add=True`: Automatically set to the current date and time when the object is first created.
- **updated_at**: `DateTimeField`
  - `auto_now=True`: Automatically set to the current date and time every time the object is saved.

## 2. Calendar Model

Represents a calendar entity, which can contain multiple events.

In [ ]:
class Calendar(TimeStampedModel):
    """Model representing a calendar."""
    title = models.CharField(max_length=200)

    def __str__(self) -> str:
        return self.title

### Properties:
- **title**: `CharField` (max_length=200)
  - The name or title of the calendar.

### Methods:
- **__str__**: Returns the `title` of the calendar.

## 3. Category Model

Represents a category for events, allowing for classification and color-coding.

In [ ]:
class Category(TimeStampedModel):
    name = models.CharField(max_length=100)
    color = models.CharField(max_length=7, blank=True, help_text="Hex color code")
    description = models.TextField(blank=True)

    def __str__(self) -> str:
        return self.name

### Properties:
- **name**: `CharField` (max_length=100)
  - The name of the category.
- **color**: `CharField` (max_length=7)
  - Optional hex color code (e.g., "#FF0000") for visual representation.
- **description**: `TextField`
  - Optional detailed description of the category.

### Methods:
- **__str__**: Returns the `name` of the category.

## 4. Event Model

Represents an event within a calendar. This is the core model with extensive validation logic.

In [ ]:
class Event(TimeStampedModel):
    # ... (fields defined below)

### Constants & Choices:
- **EVENT_TYPES**:
  - `SINGLE_DAY` ('single'): For events occurring on a single date.
  - `MULTI_DAY` ('multi'): For events spanning multiple dates.
- **EVENT_STATUS**:
  - `EVENT_STATUS_DRAFT` ('draft'): Event is not yet published.
  - `EVENT_STATUS_PUBLISHED` ('published'): Event is visible/active.

### Fields:
- **category**: `ForeignKey` to `Category`
  - Protected deletion (`on_delete=models.PROTECT`).
- **calendar**: `ForeignKey` to `Calendar`
  - Cascading deletion (`on_delete=models.CASCADE`).
- **title**: `CharField` (max_length=200)
  - Title of the event.
- **description**: `TextField`
  - Optional description.
- **organizer**: `CharField` (max_length=100)
  - Optional name of the organizer.
- **location**: `CharField` (max_length=255)
  - Optional location string.
- **slug**: `SlugField`
  - Unique URL-friendly identifier. Automatically generated from title if not provided.
- **type**: `CharField`
  - Choice of 'single' or 'multi'. Defaults to 'single'.
- **start_date**: `DateField`
  - Required for multi-day events. Should be null for single-day events (logic enforced in `clean`).
- **end_date**: `DateField`
  - Required for multi-day events.
- **start_time**: `TimeField`
  - Required start time.
- **end_time**: `TimeField`
  - Required end time.
- **entry_form_required**: `BooleanField`
  - Defaults to `False`.
- **reminder_enabled**: `BooleanField`
  - Defaults to `False`.
- **status**: `CharField`
  - Choice of 'draft' or 'published'. Defaults to 'draft'.

### Validation Logic (`clean` method):
The `clean` method enforces strict consistency rules:
1. **Time Validation**: `end_time` must be after `start_time`.
2. **Multi-day Events**:
   - Must have both `start_date` and `end_date`.
   - `end_date` must be after `start_date`.
3. **Single-day Events**:
   - Should NOT have `start_date` or `end_date` set (assuming date is handled by the calendar context or a separate field not shown, or this logic implies single day events might rely purely on time or a specific date field if it existed, but based on the code: `if self.start_date or self.end_date: raise ValidationError`).
   
   *Note: It seems single-day events might be missing a specific 'date' field in this model definition, or they rely on `start_date` being used differently. Based on the code `if self.type == self.SINGLE_DAY: if self.start_date or self.end_date: raise...`, it strictly forbids these fields for single day events.*

In [ ]:
    def clean(self):
        if self.start_time and self.end_time and self.end_time <= self.start_time:
            raise ValidationError("End time must be after start time.")
        elif not self.start_time or not self.end_time:
            raise ValidationError("Start time and end time are required.")
        
        if self.type == self.MULTI_DAY:
            if not self.start_date or not self.end_date:
                raise ValidationError("Start date and end date are required for multi-day events.")
            if self.start_date >= self.end_date:
                raise ValidationError("End date must be after start date.")
        elif self.type == self.SINGLE_DAY:
            if self.start_date or self.end_date:
                raise ValidationError("Start date and end date should be empty for single day events.")

### Slug Generation (`save` method):
Automatically generates a unique slug from the title if one isn't provided. Handles duplicates by appending a counter (e.g., `my-event-1`).

In [ ]:
    def save(self, *args, **kwargs):
        if not self.slug and self.title:
            base_slug = slugify(self.title)
            slug = base_slug
            counter = 1
            while Event.objects.filter(slug=slug).exists():
                slug = f"{base_slug}-{counter}"
                counter += 1
            self.slug = slug
        super().save(*args, **kwargs)